In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [10]:
# Definir la región de interés
roi = ee.FeatureCollection('projects/facub-gee/assets/Nepal')
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
# Añadir Google Satellite como mapa base
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

In [3]:
# Parámetros de visualización para imágenes multiespectrales en falso color nir-swir1-swir2
visFalseColor = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B11', 'B12']
}

# Parámetros de visualización para la biomasa
visBiomass = {'min': 0, 'max': 100, 'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']}

# Parámetros de visualización para Sentinel-1
visParamsS1 = {
    'min': -25,
    'max': 0,
    'bands': ['VV', 'VH']
}

# Definir el año de análisis
year = 2020

# Definir la fecha de inicio y fin para filtrar la colección
start = f'{year}-01-01'
end = f'{year}-12-31'

In [4]:

# Filtrar la colección Sentinel-2
s2Col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start, end)

# Filtrar la colección Sentinel-2 cloud score
cloudCol = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(roi) \
    .filterDate(start, end)

# Combinar la colección, máscara de nubes y hacer un composite mediano
def mask_clouds(image):
    cloud_score = ee.Image(cloudCol.filterMetadata('system:index', 'equals', image.get('system:index')).first())
    mask = cloud_score.select('probability').lt(10)
    return image.updateMask(mask)

image = s2Col.map(mask_clouds).median().toFloat().clip(roi)

# Calcular NDVI y MNDWI
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
# Calcular NDBI
ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')

# Añadir bandas NDVI, MNDWI y NDBI a la imagen
image = image.addBands(ndvi).addBands(mndwi).addBands(ndbi)

# Integrar la banda "label" de Dynamic World
dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dwLabel = dwCol.select('label').median().clip(roi)


# Añadir la banda "label" de Dynamic World a la imagen
image = image.addBands(dwLabel.rename('label'))

# Cargar la imagen de altura de dosel
canopy_height = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1")

# Recortar la imagen de altura de dosel a la ROI
canopy_height = canopy_height.clip(roi)

# Agregar la banda de altura de dosel a la imagen de datos de biomasa de GEDI
image = image.addBands(canopy_height.rename('CH'))



# Definir las bandas para predecir la biomasa
bands = ['B11', 'B12', 'B2', 'B3', 'B4', 'CH', 'MNDWI', 'NDBI', 'NDVI', 'RVI', 'VH', 'VV', 'VV_VH_div_2', 'VV_div_VH', 'VV_minus_VH', 'VV_plus_VH', 'elevation', 'slope']


# # Función de corrección con el ángulo de incidencia
# def apply_incidence_angle_correction(image):
#     angle = image.select('angle')
#     sigma0 = image.select(['VV', 'VH', 'HH', 'HV'])
#     correction_factor = ee.Image.constant(10).pow(angle.divide(ee.Image.constant(90)).log10())
#     corrected_sigma0 = sigma0.divide(correction_factor)
#     return image.addBands(corrected_sigma0, None, True)


# Función de preprocesamiento
def preprocess(image):
    # Aplicar corrección de ángulo de incidencia
    # image = apply_incidence_angle_correction(image)
    # Filtrado de Speckle
    speckle_filtered = image.focal_median(kernelType='circle', radius=50, units='meters')

    return speckle_filtered
# Definir la función de máscara de borde
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

# Filtrar y procesar la colección Sentinel-1 para VV
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate(start, end)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')

    .map(mask_edge)
    .map(preprocess)
)

# Filtrar y procesar la colección Sentinel-1 para VH
img_vh = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate(start, end)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VH')

    .map(mask_edge)
    .map(preprocess)
)


# Promedio de imágenes VV y VH
s1_vv = img_vv.median().rename('VV')
s1_vh = img_vh.median().rename('VH')



# Calcular el índice RVI
rvi = s1_vh.divide(s1_vv.add(s1_vh)).multiply(4).rename('RVI')
vv_plus_vh = s1_vv.add(s1_vh).rename('VV_plus_VH')
vv_minus_vh = s1_vv.subtract(s1_vh).rename('VV_minus_VH')
vv_div_vh = s1_vv.divide(s1_vh).rename('VV_div_VH')
vv_vh_div_2 = s1_vv.add(s1_vh).divide(2).rename('VV_VH_div_2')

# Combinar las bandas VV, VH y RVI en una sola imagen
s1_combined = ee.Image.cat([s1_vv, s1_vh, vv_plus_vh, vv_minus_vh, vv_div_vh, vv_vh_div_2, rvi]).clip(roi)

# Añadir las bandas VV, VH, RVI y álgebra de bandas a la imagen
image = image.addBands(s1_combined)

# Añadir SRTM DEM y pendiente
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)

# Añadir las bandas DEM y pendiente a la imagen
image = image.addBands(dem.rename('elevation'))
image = image.addBands(slope)

# Seleccionar solo las bandas especificadas
image = image.select(bands)

In [5]:
feature_names = ['B11', 'B12', 'B2', 'B3', 'B4', 'CH', 'MNDWI', 'NDBI', 'NDVI', 'RVI', 'VH', 'VV', 'VV_VH_div_2', 'VV_div_VH', 'VV_minus_VH', 'VV_plus_VH', 'elevation', 'slope']
label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/Rf_Gee_nepal_2"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [14]:
# Reducir la imagen clasificada a su valor máximo en la región de interés
max_value = classified.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=roi,
    scale=100,
    maxPixels=1e9
)

# Obtener el valor máximo de la reducción
max_agbd = max_value.get('classification').getInfo()
print(f"Max value of the classified image: {max_agbd}")

Max value of the classified image: 155.93490694999983


In [6]:
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.centerObject(roi)

visPredictedBiomass = {
    'min': 0,
    'max': 60,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}

Map = geemap.Map(zoom=11)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

Map.addLayer(
    classified,
    visPredictedBiomass,
    "classification",)
Map.add_colorbar(visPredictedBiomass, label="Predicted Biomass")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [15]:
import time

# Parámetros de exportación
task = ee.batch.Export.image.toDrive(
    image=classified,
    description='Nepal_2020_3',
    folder='EE_RF_3',
    region=roi,
    scale=30,
    crs='EPSG:32644',  # CRS adecuado para Nepal
    maxPixels=1e13
)
task.start()

print("Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.")

# Esperar a que la tarea de exportación se complete (opcional)
while task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

if task.status()['state'] == 'COMPLETED':
    print('Exportación completada con éxito.')
else:
    print('Error en la exportación:', task.status())

Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación completada con éxito.
